# Verkeersdata Analyse Porfolio

In [1]:
# Libraries inladen
import pandas as pd
pd.set_option('display.max_columns', None) # Als er veel kolommen zijn, wil ik ze allemaal zien bij printen
import matplotlib.pyplot as plt
import requests

In [ ]:
# Datasets inladen
accidents = pd.read_csv("datasets/accident_data.csv")
roads = pd.read_csv("datasets/road_data.csv")
weather = pd.read_csv("datasets/weather_data.csv")

## Datasets mergen aan elkaar

Eerst een kijke nemen op de datasets:

In [ ]:
# Overzicht Data
def overzicht(df, name):
    print(f"\n {name} \n")
    print("-------------------------")
    print(df.head())

overzicht(accidents, "Accident Data")
overzicht(weather, "Weather Data")
overzicht(roads, "Road Data")

Accidents en weather data mergen

In [ ]:
# Start_Time kolom komt 2 keer voor (bij accidents en weather).
weather.drop(columns=["Start_Time"], inplace=True)

# Mergen op basis van het aantal rijen
merged = pd.concat([accidents, weather], axis=1)

#Controleren
merged.head()

Accidents en road data mergen

In [ ]:
# Omdat ik al een merged data heb zal ik die data gebruiken
df = pd.merge(merged, roads, on='ID', how='left')

#### Kijken of alles goed is gegaan

In [ ]:
print(accidents.shape[0])
print(weather.shape[0])
print(roads.shape[0])
print(df.shape[0])

## Onvolledige data verwijderen of aanvullen

In [ ]:
# Overzicht Data
print(df)

In [ ]:
# Percentage van missende waarden per kolom
print((df.isnull().sum() / len(df) * 100))

In de kolommen End_lat en End_lng zitten ook heel veel missende waardes. Aangezien ik al de coordinaten van de incident locaties heb (Start_Lat - Start_Lng) heb ik deze twee kolommen niet nodig.

In [ ]:
# End_Lat en End_Lng kolommen verwijderen met  veel missende waardes (niet nodig)
df.drop(['End_Lat', 'End_Lng'], axis=1, inplace=True)

In de description en city kolommen zitten heel weininig nan waardes, hierbij kunnen we heel makkelijk deze rijen veranderen, want het zal geen effect hebben op de analyse.

In [ ]:
# Rijen met nan waardes verwijderen
df.dropna(subset=["Description", "City", "Zipcode"], inplace=True)

In de kolom 'Precipitation(in)' zitten veel missende waarden. Aangezien waarschijnlijk in de meeste geval niet regenen of sneeuwen betekent dat er geen regen of sneeuw is gevallen, besloot ik om de missende waarden met '0.0' te invullen. Zo heb ik geen dataverlies en kan werken met deze kolom.

In [ ]:
# None waardes veranderen met 0
df.fillna({"Precipitation(in)": 0.0}, inplace=True)

Als je kijkt naar de Wind_Chill(F) kolom, zie je dat het meestaal heel dichtbij of soms helemaal hetzelfde is met de Temperature(F) kolom. Dit betekent dat de NaN waardes gewoon met de waardes van Temperature(F) opgevuld kan worden. Dit geldt ook voor de kolom Temperature(F).

In [ ]:
df[["Wind_Chill(F)","Temperature(F)"]].head()

In [ ]:
# Opvullen met dichtstbijzijnde waardes
df.fillna({"Wind_Chill(F)": "Temperature(F)"}, inplace=True)
df.fillna({"Temperature(F)": "Wind_Chill(F)"}, inplace=True)

Omdat de resterende kolommen heel weinig NaN waardes bevatten, kunnen de rijen met de NaN values gelijk werwijderd kan worden. Dit heeft geen effect op de analyse en de data begrijpelijker.

In [ ]:
# De rijen met weinig missende waardes verwijderen
df.dropna(subset=["Humidity(%)", "Pressure(in)" 
                       ,"Visibility(mi)", "Wind_Direction", 
                       "Wind_Speed(mph)", "Weather_Condition", "Sunrise_Sunset"], 
               inplace=True)

In [ ]:
df.isnull().sum()

Alle missende waardes zijn nu verwijderd.

## Onnodige Kolommen verwijderen
Om de data overzichtelijk te houden, onnodig geheugengebruik te voorkomen en alleen met kolommen te werken die relevant zijn voor de analyse, is het handig om een paar overbodige kolommen te verwijderen.

In [ ]:
# Accidents kolommen
df.drop(columns=[
    "Description", "Source", "Zipcode", "Distance(mi)"
]
               , inplace=True)

# Weather kolommen
df.drop(columns=[
    "Wind_Chill(F)", "Visibility(mi)", "Wind_Direction", "Weather_Timestamp"	 
]
            , inplace=True)

In [ ]:
print(df.columns)

## Onrealistische waarden filtreren
Uitschieters of fout waardes verwijderen.

In [ ]:
# Kijken of er foute waarden tussen zitten
df.describe()

#### Wind-luchtdruk (Pressure)
Winddruk kan niet minder dan 25 of meer dan 32 zijn. Meer informatie kun je [hier](https://www.rovary.com/pages/luchtdruk) vinden. Daarom filtreer ik de waarden in Pressure kolom tussen 25 en 32 inch.

In [ ]:
# Waarden filtreren tussen 25 en 32
df = df[(df['Pressure(in)'] >= 25) & (df['Pressure(in)'] <= 32)]

#### Windsnelheid (Wind_speed)
Volgens de [Saffir-Simpson orkaanschaal](https://www.nhc.noaa.gov/aboutsshws.php) beginnen orkanen bij windsnelheden van 74 mph. Zelfs de zwaarste orkanen (categorie 5) hebben windsnelheden van 157 mph of hoger. Dus waarden zoals 822mph zijn fysiek onmogelijk en dus fout. Om het realistisch te houden zal ik de windsnelheid filtreren tot 120 mph.

In [ ]:
# Windsnelheid filtreren tot 120 mph
df = df[(df['Wind_Speed(mph)'] <= 120)]

#### Controleren

In [ ]:
df.describe()

## Datatypes Corrigeren 
Elke kolom in de datasets moet het juiste datatype hebben, zodat verdere analyses soepel verlopen en foutmeldingen worden voorkomen.

### Accidents

In [ ]:
df.info()

In [ ]:
# City, COuntry en State zijn ook string type kolommen
df[['City', 'County', 'State']] = df[['City', 'County', 'State']].astype('string')

# Om datatypes te veranderen, moeten de nanoseconden in deze kolommen verwijderd worden
df['Start_Time'] = df['Start_Time'].str.extract(r'^(.{19})')[0]
df['End_Time'] = df['End_Time'].str.extract(r'^(.{19})')[0]

# Start_Time en End_Time moeten worden omgezet naar datetime
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])

# Temperature moet float worden
df['Temperature(F)'] = pd.to_numeric(df['Temperature(F)'], errors='coerce')

# Weather_Condition en Sunrise_Sunset zijn ook strings
df[['Sunrise_Sunset', 'Weather_Condition']] = df[['Sunrise_Sunset', 'Weather_Condition']].astype('string')

### Laatste check

In [ ]:
print(df.info())

Alles ziet er goed uit

## Aantal ongevallen per tijdstippen
In deze stap analyseer ik op **welke dagen** en in **welke maanden** de meeste verkeersongevallen plaatsvinden.

We gebruiken hiervoor de `Start_Time` kolom in de dataset. Hieruit maak ik 2 nieuwe kolommen aan:
- de **naam van de dag** (zoals maandag, dinsdag)  
- de **naam van de maand** (zoals januari, februari)

Daarna visualiseer ik de spreiding van ongevallen over de week en het jaar. Deze inzichten helpen ons om **drukke dagen** en **seizoensgebonden patronen** te herkennen.

In [ ]:
# Nieuwe kolommen maken uit Start_Time 
df['Dag'] = df['Start_Time'].dt.day_name()
df['Maand'] = df['Start_Time'].dt.month_name()

# Controleren
df.head()

In [ ]:
plt.figure(figsize=(10, 5))

# Week kolom maken en de dagen op een volgorde zetten
df['Dag'].value_counts().loc[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].plot(kind='bar')

plt.title("Aantal Ongevallen per Dag van de Week")
plt.xlabel("Dag van de Week")
plt.ylabel("Aantal Ongevallen")

plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

plt.show()

De meeste verkeersongevallen vinden plaats op **vrijdag**, gevolgd door **donderdag** en **woensdag**. Dit patroon wijst erop dat ongevallen frequenter zijn aan het einde van de werkweek. Mogelijke verklaringen hiervoor zijn:

	•	Meer verkeer op vrijdag vanwege mensen die eerder naar huis willen of op reis gaan voor het weekend.
	•	Vermoeidheid en stress na een drukke werkweek kunnen bijdragen aan verminderde concentratie op de weg.
	•	Zaterdag en zondag laten aanzienlijk minder ongevallen zien, wat logisch is gezien de lagere verkeersdrukte in het weekend.

Deze inzichten kunnen nuttig zijn bij het plannen van verkeersmaatregelen of voorlichting, vooral gericht op de risicovolle werkdagen.

In [ ]:
plt.figure(figsize=(10, 5))

# Maand kolom maken en maanden op een volgorde zetten
df['Maand'].value_counts().loc[['January','February','March','April','May','June','July','August','September','October','November','December']
].plot(kind='bar', color='mediumseagreen')

# Visualiseren
plt.title("Aantal Ongevallen per Maand")
plt.xlabel("Maand")
plt.ylabel("Aantal Ongevallen")

plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

plt.show()

Het hoogste aantal ongevallen vindt plaats in **december**, gevolgd door **januari** en **november**.In de zomermaanden (zoals juli en augustus) ligt het aantal ongevallen juist relatief laag. Mogelijke verklaringen:

	•	Wintermaanden brengen slechtere weersomstandigheden zoals sneeuw, regen en gladheid.
	•	December heeft extra verkeer door feestdagen en drukte op de wegen.
	•	In de zomer zijn er vakanties en minder woon-werkverkeer, wat mogelijk leidt tot minder incidenten.

Dit inzicht toont aan dat seizoen en weersomstandigheden een duidelijke invloed hebben op verkeersveiligheid.

## Verdeling van ongevallen per dagdeel
In deze stap onderzoek ik in welk deel van de dag de meeste verkeersongelukken plaatsvinden.  
Ik zal de dag verdelen in vier periodes zodat ik kan analyseren. Hiervoor zal ik gebruiken maken van `Start_Time`:

- Nacht (00:00 – 06:00)
- Ochtend (06:00 – 12:00)
- Middag (12:00 – 18:00)
- Avond (18:00 – 00:00)

Deze indeling helpt ons inzicht te krijgen in wanneer het verkeer het gevaarlijkst is. Bijv. tijdens iedereen naar werkt gaat of terug komt van werk.

In [ ]:
# Een "Uur" kolom maken uit `Start_Time` kolom
df['Uur'] = pd.to_datetime(df['Start_Time']).dt.hour

# Dag verdelen in vier
def dag_verdelen(uur):
    if uur >= 0 and uur < 6:
        return 'Nacht'
    elif uur >= 6 and uur < 12:
        return 'Ochtend'
    elif uur >= 12 and uur < 18:
        return 'Middag'
    else:
        return 'Avond'

df['Dagdeel'] = df['Uur'].apply(dag_verdelen)

# Controleren
df.head()

In [ ]:
plt.figure(figsize=(8, 5))

# Dagdeel kolom aanmaken
df['Dagdeel'].value_counts().loc[['Nacht', 'Ochtend', 'Middag', 'Avond']].plot(kind='bar', color='slateblue')

# Visualiseren
plt.title("Aantal Ongevallen per Dagdeel")
plt.xlabel("Dagdeel")
plt.ylabel("Aantal Ongevallen")

plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()

plt.show()

Uit de grafiek blijkt dat de meeste ongevallen plaatsvinden in de **middag**, gevolgd door de **ochtend**.  
Dit zou verklaard kunnen worden door druk verkeer tijdens werkuren en spitsmomenten. Mensen rijden nog sneller en zorgeloos om ergens op tijd te zijn. Dat veroorzaakt ook meer incidenten. 

Opvallend is dat het aantal ongevallen in de **nacht** het laagst is, waarschijnlijk door minder verkeer op de weg.  
Deze inzichten kunnen handig zijn voor het bepalen van momenten waarop extra toezicht of verkeersveiligheidsmaatregelen nodig zijn en waneer wij als mensen meer voorzichtig moeten rijden op de weg.

## Analyse van het ongeval ratio per stad
Om een dieper inzicht te krijgen in verkeersveiligheid in verschillende steden, was het niet voldoende om enkel het absolute aantal verkeersincidenten per stad te analyseren. Een stad met veel inwoners zal namelijk logischerwijs meer ongevallen hebben, simpelweg door de grotere bevolkingsomvang. Dit geeft een vertekend beeld bij het vergelijken van steden met uiteenlopende inwoneraantallen.

Daarom heb ik ervoor gekozen om een aanvullende dataset in te laden met de bevolkingsgrootte per stad in de VS. Deze data heb ik verkregen via een publieke API van OpenDataSoft (us-cities-demographics). Door deze populatiegegevens te combineren met het aantal ongevallen per stad, kon ik een incidentratio per 100.000 inwoners berekenen.

Met deze ratio kan ik:

	•	Steden eerlijker vergelijken, ongeacht de grootte.
	•	Inzicht krijgen in welke steden relatief het gevaarlijkst zijn.
	•	Specifieke patronen of afwijkingen ontdekken die niet zichtbaar waren bij alleen het totaal aantal incidenten.

Kortom: de aanvullende bevolkingsdataset is essentieel om de verkeersveiligheid per stad in verhouding tot het aantal inwoners te analyseren. Alleen zo kunnen we echt zinvolle inzichten verkrijgen.

In [ ]:
# API aanvragen
url = "https://public.opendatasoft.com/api/records/1.0/search/"
params = {
    "dataset": "us-cities-demographics",
    "rows": 100,
    "start": 0
}

all_data = []

# Hele data inladen
while params["start"] < 2900:  # Totaal is 2981
    r = requests.get(url, params=params)
    data = r.json()
    
    for record in data["records"]:
        all_data.append(record["fields"])
    
    params["start"] += params["rows"]

# Maak een DataFrame met de benodigde kolommen
df1 = pd.DataFrame(all_data)[["city", "state_code", "total_population"]]
df1

In [ ]:
# Een aparte DataFrame maken
ongeval_steden = df["City"].value_counts().reset_index()
ongeval_steden.rename(columns={"City":"city"}, inplace=True)

# Dubbels verwijderen
df1.drop_duplicates(subset=['city'], inplace=True)

# Mergen met de externe API
df_merged_city = pd.merge(ongeval_steden, df1, on=['city'], how="inner")
df_merged_city

In [ ]:
plt.figure(figsize=(12, 6))

# Rate kolom per 100.000 inwoner
df_merged_city["incident_rate"] = (df_merged_city["count"] / df_merged_city["total_population"]) * 100_000
top_20 = df_merged_city.sort_values(by="incident_rate", ascending=False).head(20)

# Visualiseren
plt.bar(top_20["city"], top_20["incident_rate"], color="tomato")
plt.title("Top 20 Steden met Hoogste Verkeersongevalratio's")
plt.xlabel("Stad")
plt.ylabel("Incidenten per 100.000 inwoners")

plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)

plt.show()

In de bovenstaande grafiek tonen we de 20 steden met de hoogste ratio van verkeersongevallen per 100.000 inwoners. Opvallend is dat er zowel grote steden als middelgrote tot kleine steden in deze lijst staan. Enkele opvallende punten:

	•	Miami, Greenville, Richmond en Orlando hebben de hoogste ratio’s, ondanks dat sommige van deze steden relatief minder inwoners hebben. Dit betekent dat het risico op een verkeersongeval in verhouding tot de bevolking daar erg hoog is.
	•	Steden zoals Baton Rouge en Flint staan ook in de top, wat mogelijk wijst op structurele verkeersproblemen of lage infrastructuurkwaliteit.
	•	Interessant is dat steden zoals Springfield en Rochester, die minder bekend zijn als drukke verkeersknooppunten, toch hoog eindigen in verhouding tot hun populatie.
	•	Steden als Charlotte, Atlanta en Sacramento zijn iets groter, maar hebben relatief lagere ratio’s dan de rest van de top, wat betekent dat hun verkeer iets beter beheerd wordt of dat er betere infrastructuur is.

## Analyse van het ongeval ratio per staat
Om de verkeersongevallen per staat op een eerlijke manier te vergelijken, is het belangrijk om rekening te houden met de bevolkingsgrootte. Aangezien ik alleen de populatiegegevens per stad heb, heb ik per staat de totale populatie berekend door de populatie van alle steden in die staat bij elkaar op te tellen.

Zo kan ik het aantal ongevallen per 100.000 inwoners per staat bepalen en zo zien welke staten relatief het zwaarst getroffen zijn, los van hun grootte.

In [ ]:
# Totale populatie berekenen per staat
populatie_staat = df_merged_city.groupby("state_code")["total_population"].sum()

# Totaal ongevallen berekenen per staat
ongevallen_staat = df_merged_city.groupby("state_code")["count"].sum()

# Mergen
df_staat = pd.merge(populatie_staat, ongevallen_staat, on=["state_code"], how="inner")

# Incidenten berekenen per 100.000 
df_staat["Ongeval ratio"] = (df_staat["count"] / df_staat["total_population"]) * 100_000

# Selecteren top 10
top_10 = df_staat.nlargest(10, "Ongeval ratio")

In [ ]:
# Visualisatie
plt.figure(figsize=(12, 6))
plt.bar(top_10.index, top_10["Ongeval ratio"], color="tomato")

plt.title("Top 10 Staten met Hoogste Ongevallenratio per 100.000 Inwoners")
plt.xlabel("Staat")
plt.ylabel("Ongevallen per 100.000 inwoners")

plt.grid(True)
plt.tight_layout()

plt.show()

De grafiek toont dat North Carolina (NC), Louisiana (LA) en Minnesota (MN) de hoogste ongevalratio’s hebben, ondanks dat sommige van deze staten niet tot de grootste behoren qua inwonertal.

Mogelijke verklaringen:

	•	Slechtere verkeersveiligheid of infrastructuur
	•	Weersomstandigheden of rijgedrag
	•	Specifieke stedelijke gebieden met veel verkeer binnen een relatief kleine populatie

Deze analyse geeft beleidsmakers en onderzoekers richting bij het identificeren van staten waar verkeersveiligheid extra aandacht verdient..

In [ ]:
asd